In [1]:
%matplotlib notebook

In [2]:
import numpy as np
import pandas as pd
import torch, IPython, itertools, string
import random, time, warnings
import matplotlib.pyplot as plt
from matplotlib import animation

In [3]:
warnings.simplefilter('ignore')

In [4]:
class Point():
    def __init__(self, linkage, name, at=None, on=None, anchored_at=None):
        self.linkage = linkage
        self.name = name
        if at is not None:
            self.mode = 'at'
            self.type = 'free_point'
            self.parent = None
            self.r = torch.tensor(at).to(torch.float)
        elif on is not None:
            self.mode = 'on'
            self.type = 'co_point'
            self.parent = on
            if self.parent.__class__.__name__ is 'Point':
                self.r = self.parent.r
            elif self.parent.__class__.__name__ is 'Line':
                raise Exception()
        elif anchored_at is not None:
            self.mode = 'anchored_at'
            self.type = 'anchored_point'
            self.parent = None
            self.r = torch.tensor(anchored_at).to(torch.float)
        else:
            self.mode = None
            self.type = 'null_point'
            self.parent = None
            self.r = torch.tensor(anchored_at).to(torch.float)
    
    '''
    @property
    def r(self):
        return(None)
    '''
    
    def __repr__(self):
        if self.mode is 'at':
            return('Point_{}(at={})'.format(self.name, str(self.r.tolist())))
        elif self.mode is 'on':
            return('Point_{}(on={})'.format(self.name, str(self.parent)))
        elif self.mode is 'anchored_at':
            return('Point_{}(anchored_at={})'.format(self.name, str(self.r.tolist())))
        else:
            return('Point_{}(None)'.format(self.name))
    
    def constrain_on(self, parent):
        self.mode = 'on'
        self.type = 'co_point'
        self.parent = parent
        if self.parent.__class__.__name__ is 'Point':
            self.r = self.parent.xyz
        elif self.parent.__class__.__name__ is 'Line':
            raise Exception()

In [5]:
class Line():
    def __init__(self, linkage, name, r1, r2):
        self.linkage = linkage
        self.name = name
        self.target_length = None
        if r1.__class__.__name__ is 'Point':
            self.p1 = Point(self.linkage, '{}{}'.format(self.name, '1'), on=r1)
        else:
            self.p1 = Point(self.linkage, '{}{}'.format(self.name, '1'), at=r1)
        if r2.__class__.__name__ is 'Point':
            self.p2 = Point(self.linkage, '{}{}'.format(self.name, '2'), on=r2)
        else:
            self.p2 = Point(self.linkage, '{}{}'.format(self.name, '2'), at=r2)
        
    def __repr__(self):
        return('Line_{}(p1={}, p2={})'.format(self.name, str(self.p1), str(self.p2)))
    
    def length(self):
        L = (self.p2.r-self.p1.r).pow(2).sum().pow(0.5)
        return(L)
    
    def constrain_length(self, L):
        self.target_length = L
        self.linkage.update()

In [6]:
class Linkage():
    def __init__(self):
        
        self.points = {}
        self.anchors = {}
        self.lines = {}
    
        self.names = {}
        for _type in ['point', 'anchor', 'line']:
            self.names[_type] = []
            letters = string.ascii_letters[-26:]
            if _type is 'line':
                letters = letters.lower()
            for n in range(3):
                for t in itertools.product(letters, repeat=n):
                    self.names[_type].append(''.join(t))
            self.names[_type] = iter(self.names[_type][1:])
    
    def add_point_at(self, r):
        name = next(self.names['point'])
        self.points[name] = Point(self, name, at=r)
        
    def add_point_on(self, p):
        name = next(self.names['point'])
        self.points[name] = Point(self, name, on=p)
    
    def add_anchor_at(self, r):
        name = next(self.names['anchor'])
        self.anchors[name] = Point(self, name, at=r)
    
    def add_line(self, r1, r2):
        name = next(self.names['line'])
        self.lines[name] = Line(self, name, r1, r2)
        
    @property
    def N(self):
        N = 0
        N += len(self.points)
        N += len(self.anchors)
        N += 2*len(self.lines)
        return(N)
        
    def xyz(self):
        xyz = torch.zeros((self.N, 3), requires_grad=True)
        return(xyz)
        
    def energy(self):
        E = 0.0
        return(E)
            
    def update(self):
        xyz = self.xyz()
        optimizer = torch.optim.SGD([xyz], lr=0.01, momentum=0.99, dampening=0.99)
        for epoch in range(1000):
            optimizer.zero_grad()
            E = self.energy()
            E.backward()
            optimizer.step()
            #mechanism_plot.E_list.append(E.item())
            if epoch % 1 == 0:
                #mechanism_plot.update()
                print(epoch, E.item())
                time.sleep(0.01)

In [7]:
linkage = Linkage()

In [8]:
linkage.add_anchor_at([0,0,0])

In [9]:
linkage.anchors

{'A': Point_A(at=[0.0, 0.0, 0.0])}

In [10]:
linkage.add_line(linkage.anchors['A'],[2,2,1])

In [11]:
linkage.lines

{'a': Line_a(p1=Point_a1(on=Point_A(at=[0.0, 0.0, 0.0])), p2=Point_a2(at=[2.0, 2.0, 1.0]))}

In [12]:
linkage.lines['a'].p1

Point_a1(on=Point_A(at=[0.0, 0.0, 0.0]))

In [13]:
linkage.lines['a'].p2

Point_a2(at=[2.0, 2.0, 1.0])

In [14]:
linkage.lines['a'].length()

tensor(3.)

In [15]:
linkage.lines['a'].constrain_length(1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
zz

NameError: name 'zz' is not defined

In [ ]:
class Mechanism(torch.nn.Module):
    def __init__(self):
        super(Mechanism, self).__init__()
        self.origin = torch.tensor([0,0,0])
        self.r = torch.nn.Parameter(torch.randn((3,2,3)))
        self.N = len(self.r)
        #self.v = torch.zeros((3,2,2))
        #self.m = torch.ones((3,2,2))
        #self.T = 0.01
        #self.dt = 0.005
        #self.mu = 1
        
    def dist(self, p1, p2):
        d = (p2-p1).pow(2).sum().pow(0.5)
        return(d)
    
    def R(self):
        R = torch.zeros_like(self.r)
        R[0][0] += self.origin
        R[0][1] += self.r[0][1]
        R[1][0] += self.r[0][1]
        R[1][1] += self.r[1][1]
        R[2][0] += self.origin
        R[2][1][0] += self.r[2][1][0]
        R[2][1][1] += self.r[2][0][1]
        return(R)
    
    def energy(self):
        # constraints
        # L0 has length 1
        # L1 has length 3
        # L2 has length 4
        # L00 on origin and 
        # L01 on L10
        # L11 on L2 --> length(L11,L20) + length(L11,L21) = length(L20,L21)
        # L20 on origin
        # L2 horizontal --> L201 = L211
        
        '''
        E = 0
        E += (self.dist(self.r[0][0],self.r[0][1])-1).pow(2)
        E += (self.dist(self.r[1][0],self.r[1][1])-3).pow(2)
        E += (self.dist(self.r[2][0],self.r[2][1])-4).pow(2)
        E += (self.dist(self.r[0][0],self.origin)-0).pow(2)
        E += (self.dist(self.r[0][1],self.r[1][0])-0).pow(2)
        E += (self.dist(self.r[1][1],self.r[2][0])+self.dist(self.r[1][1],self.r[2][1])-self.dist(self.r[2][0],self.r[2][1])).pow(2)
        E += (self.dist(self.r[2][0],self.origin)-0).pow(2)
        E += (self.r[2][0][1]-self.r[2][1][1]).pow(2)
        '''
    
        #r = self.r + 0*torch.randn_like(self.r)
        
        r = self.R()
    
        E = 0
        E += (self.dist(r[0][0],r[0][1])-1).abs()
        E += (self.dist(r[1][0],r[1][1])-3).abs()
        E += (self.dist(r[2][0],r[2][1])-4).abs()
        #E += (self.dist(r[0][0],self.origin)-0).abs()
        #E += (self.dist(r[0][1],r[1][0])-0).abs()
        E += self.dist(r[1][1],r[2][0]).abs()+self.dist(r[1][1],r[2][1]).abs()-self.dist(r[2][0],r[2][1]).abs()
        #E += (self.dist(r[2][0],self.origin)-0).abs()
        #E += (self.dist(r[2][0],r[0][0])-0).abs() # L20 on L00
        E += (r[2][0][1]-r[2][1][1]).abs()
        E += r[:,:,2].abs().sum()

        return(E)
    
    def theta(self):
        u1 = torch.tensor([1,0])
        r2 = r[0][1]-r[0][0]
        u2 = r2/(r2.pow(2).sum().pow(0.5))
        embed()
        cos_theta = (u1*u2).sum()
        theta = torch.arccos(cos_theta)
        return(theta)
       
    '''
    def update(self, method):
        if method == 'MC':
            E_initial = energy(r)
            n = random.randint(0,2)
            m = random.randint(0,1)
            rand_disp = 0.01*(2*torch.rand(2)-1)
            r[n][m] += rand_disp
            E_final = energy(r)
            if E_final < E_initial:
                return(E_final)
            elif random.random() < torch.exp(-(E_final-E_initial)/T):
                return(E_final)
            else:
                r[n][m] -= rand_disp
                return(E_initial)
        elif method == 'GD':
            self.optimizer.zero_grad()
            E = self.energy(self.r)
            #F = -1.0*torch.autograd.grad(E, self.r, create_graph=False)[0]
            #F -= self.mu*self.v
            #a = F/self.m
            #self.r = self.r + self.v*self.dt
            #self.r = self.r + a*self.dt
            #self.v = self.v + a*self.dt
            E.backward()
            self.optimizer.step()
            return(E)
        else:
            raise Exception('Options for method: MC, GD')
        '''

In [ ]:
class MechanismPlot():
    def __init__(self, mechanism):
        self.mechanism = mechanism
        self.E_list = [10000.0]
        #self.dlnE_list = []
        #self.lr_list = []
        
        # Set up figure and axis
        self.size = 5
        self.fig = plt.figure(figsize=(2*self.size,self.size))
        self.ax1 = self.fig.add_subplot(121, autoscale_on=False,
            xlim=(-self.size,self.size),
            ylim=(-self.size,self.size))
        self.ax2 = self.fig.add_subplot(122, autoscale_on=False,
            xlim=(0,1),
            ylim=(0,1))
        #self.ax3 = self.ax2.twinx()
        self.ax1.set_title('Configuration')
        self.ax2.set_title('log10(E)')
        
        # Plot origin and lines
        self.ax1.scatter(
            [self.mechanism.origin[0]], [self.mechanism.origin[1]],
            marker='o', s=20, c='red', alpha=1)    
        self.lines = []
        for n in range(self.mechanism.N):
            line, = self.ax1.plot([], [], '-', markersize=3, lw=1)
            self.lines.append(line)
        self.lnE_line, = self.ax2.plot([], [], 'b-', markersize=3, lw=1, label='ln(E)')
        #self.dlnE_line, = self.ax3.plot([], [], 'r-', markersize=3, lw=1, label='dln(E)')
        time_template = ' t={:.0f}\n E={:.2f}\n T={:.5f}\n theta={:.0f}\n'
        self.time_text = self.ax1.text(0.05, 0.7, '', transform=self.ax1.transAxes)
        
    def update(self):
        r = self.mechanism.R()
        for n in range(self.mechanism.N):
            self.lines[n].set_data(
                [r[n][0][0],r[n][1][0]],
                [r[n][0][1],r[n][1][1]])
        self.lnE_line.set_xdata(torch.arange(0,len(self.E_list)))
        self.lnE_line.set_ydata(torch.log10(torch.tensor(self.E_list)))
        #self.dlnE_line.set_xdata(torch.arange(0,len(self.dlnE_list)))
        #self.dlnE_line.set_ydata(torch.tensor(self.dlnE_list))
        self.ax2.set_xlabel('Epoch')
        #self.ax2.set_ylabel('ln(energy)')
        self.ax2.set_xlim(0,len(self.E_list))
        self.ax2.set_ylim(-3,2)
        #self.ax3.set_ylim(-0.05,0.05)
        self.time_text.set_text('')
        self.fig.canvas.draw()
       
    '''
    def animate(frame):
        for k in range(1):
            E = self.mechanism.update(method='GD')
        for n in range(len(self.mechanism.r)):
            lines[n].set_data(
                [mechanism.r[n][0][0],mechanism.r[n][1][0]],
                [mechanism.r[n][0][1],mechanism.r[n][1][1]])
        time_text.set_text(time_template.format(i, E.item(), mechanism.T, 0.0))
    '''

In [ ]:
mechanism = Mechanism()

In [ ]:
mechanism_plot = MechanismPlot(mechanism)

In [ ]:
optimizer = torch.optim.SGD([mechanism.r], lr=0.01, momentum=0.99, dampening=0.99)

In [ ]:
for epoch in range(10000):
    optimizer.zero_grad()
    E = mechanism.energy()
    E.backward()
    optimizer.step()
    mechanism_plot.E_list.append(E.item())
    if epoch % 100 == 0:
        #for param_group in optimizer.param_groups:
        #    mechanism_plot.lr_list.append(param_group['lr'])
        #    print(mechanism_plot.lr_list[-1])
        #dlnE = torch.log(E) - torch.log(torch.tensor(mechanism_plot.E_list[-1]))
        #mechanism_plot.dlnE_list.append(dlnE.item())
        mechanism_plot.update()
        time.sleep(0.01)
        """
        if mechanism_plot.dlnE_list:
            dlnE = mechanism_plot.dlnE_list[-1]
            if (dlnE > -0.01) and (dlnE < 0) and (epoch % 10 == 0):
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= 0.1
            '''
            elif (dlnE > 0) and (epoch % 1 == 0):
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= 0.1 
            '''
        """

In [ ]:
'''
if False:
    anim = animation.FuncAnimation(
        fig, animate,
        frames=10000, init_func=init,
        interval=1, blit=True)
else:
    init()
    animate(0)
    animate(1)
    animate(2)
plt.show()
''';